<h1><center>ЛОГИРОВАНИЕ ЛОКАЛЬНОГО ТЕСТОВОГО ФАЙЛА</h1>

Полностью локальный сценарий<br>
Сначала мы покажем, как поднимать MLflow локально, чтобы вы могли плавно влиться в работу с сервисом. Уточним, что понятие «локальность» здесь и далее относится именно к MLflow, то есть вы будете разворачивать его на виртуальной машине по различным сценариям (собственно, локально, с базой данных, с объектным хранилищем и так далее).<br>
Теперь расскажем о необходимых шагах.<br>
1. Перейдите в вашу среду разработки на удалённом сервере. Это может быть Jupyter Notebook или его приложение в Visual Studio Code, о котором мы подробно рассказывали в уроке «Подключение к виртуальной машине». Мы рекомендуем использовать среду Jupyter, так как в ней легче работать с экспериментами.
2. Напишите shell-скрипт с названием run_mlflow_server_locally.sh для запуска Tracking Server, который будет хранить эксперименты и артефакты в папке mlflow_experiments_store.
    - Дайте старт Tracking Server, запустив ваш скрипт через консоль следующим образом: sh run_mlflow_server_locally.sh.
    - После успешного запуска через консоль вы получите примерно такое сообщение:
    - А также увидите новую папку mlflow_experiments_store в директории, из которой вы запустили shell-скрипт.
3. Не закрывая консоль, чтобы не выключить MLflow, перейдите к тестированию сервиса:
    - Создайте файл для тестового логирования. Назовите его test_artifact.txt и запишите в него test_artifact;
    - Напишите код в Jupyter Notebook, который залогирует тестовый файл.

In [1]:
# делаем import необходимых библиотек
import os

import mlflow

# устанавливаем локальное хранилище для наших экспериментов
# хранилище должно быть такое же, как и при запуске сервиса
mlflow.set_tracking_uri('file:./mlflow_experiments_store')

# получаем id эксеримента, который создаётся по умолчанию
# эксперимент по умолчанию называется Default
experiment_id = mlflow.get_experiment_by_name("Default").experiment_id

# залогируем тестовую метрику и артефакт
with mlflow.start_run(run_name='Default', experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    mlflow.log_metric("test_metric", 0)
    mlflow.log_artifact("test_artifact.txt", "test_artifact")

print(f"Run id запуска: {run_id}")

Run id запуска: b22f6fe5e95141bc9698c8794f7277bb


Проверим, что в нашем хранилище артефактов залогировалась тестовая метрика и сам артефакт:

In [2]:
!ls mlflow_experiments_store/0/{run_id}/metrics

test_metric


И вторую:

In [3]:
!cat mlflow_experiments_store/0/{run_id}/artifacts/test_artifact/test_artifact.txt

test_artifact

Аналогичное упражнение вы можете проделать и для локального запуска с базой данных SQLite. Пусть она называется mydb.sqlite, а директорию для хранения поменяйте с mlflow_experiments_store на mlflow_experiments_store_sqlite.

In [4]:
import os

import mlflow

mlflow.set_tracking_uri('http://0.0.0.0:5000')

# получаем id эксперимента, который создаётся по умолчанию
# эксперимент по умолчанию называется Default
experiment_id = mlflow.get_experiment_by_name("Default").experiment_id

with mlflow.start_run(run_name="Default", experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    mlflow.log_metric("test_metric_sqlite", 0)
    mlflow.log_artifact("test_artifact.txt", "test_artifact_sqlite")

# проверим, что наши данные сохранились в локальной папке, а также создалась база данных SQLite
assert os.path.exists("mlflow_experiments_store_sqlite")
assert os.path.exists("mydb.sqlite") 